# Prepare and configure the project.

In [ ]:
# Project settings - modify these values according to your needs.
LOAD_DATABASE_FROM_DUMP_OR_SCRAPE_OR_DO_NOTHING_FOR_TASK_1 = "SCRAPE" # "DUMP" / "SCRAPE" / "NOTHING"

# Task 1: scraping.

In [ ]:
if LOAD_DATABASE_FROM_DUMP_OR_SCRAPE_OR_DO_NOTHING_FOR_TASK_1 == "DUMP":
    from db.mysql import MysqlDAO
    db = MysqlDAO()
    db.import_database_from_dump()
    db.close()  
    
elif LOAD_DATABASE_FROM_DUMP_OR_SCRAPE_OR_DO_NOTHING_FOR_TASK_1 == "SCRAPE":
    from db.mysql import MysqlDAO
    from scraper.run_scraper import Scraper

    db = MysqlDAO()
    db.create_database()
    db.create_table()

    scraper = Scraper()
    scraper.run_spiders()

    db.close()

# Task 2: data analysis.

In [ ]:
from db.mysql import MysqlDAO
db = MysqlDAO('nekretnine')

## a) izlistati koliki je broj nekretnina za prodaju, a koliki je broj koji se iznajmljuju

In [ ]:
db.prodaja_iznajmljivanje()

## b) izlistati koliko nekretnina se prodaje u svakom od gradova (izlistati sve gradove, obuhvatiti i kuće i stanove)

### (dakle, bez soba)

In [ ]:
prodaja_po_gradovima = db.prodaja_po_gradovima()
prodaja_po_gradovima['stanovi']

In [ ]:
prodaja_po_gradovima['kuce']

In [ ]:
prodaja_po_gradovima['stanovi_i_kuce']

## c) izlistati koliko je uknjiženih, a koliko neuknjiženih kuća, a koliko stanova

In [ ]:
db.uknjizenost()

## d) prikazati rang listu prvih 30 najskupljih kuća koje se prodaju, i 30 najskupljih stanova koji se prodaju u Srbiji

In [ ]:
najskuplje = db.najskuplje()
for n in najskuplje['najskuplji_stanovi']:
    print(" | ".join(["{0}: {1}".format(k, v) for k, v in n.items()]))

In [ ]:
for n in najskuplje['najskuplje_kuce']:
    print(" | ".join(["{0}: {1}".format(k, v) for k, v in n.items()]))

## e) prikazati rang listu prvih 100 najvećih kuća i 100 najvećih stanova po površini (kvadraturi)

In [ ]:
najskuplje = db.najvece()
for n in najskuplje['najveci_stanovi']:
    print(" | ".join(["{0}: {1}".format(k, v) for k, v in n.items()]))

In [ ]:
najskuplje = db.najvece()
for n in najskuplje['najvece_kuce']:
    print(" | ".join(["{0}: {1}".format(k, v) for k, v in n.items()]))

## f) prikazati rang listu svih nekretnina izgrađenih u 2020. godini, i izlistati ih opadajuće prema ceni prodaje, odnosno ceni iznajmljivanja

In [ ]:
cene_2020 = db.cene_2020()
for n in cene_2020['prodaja']:
    print(" | ".join(["{0}: {1}".format(k, v) for k, v in n.items()]))

In [ ]:
for n in cene_2020['izdavanje']:
    print(" | ".join(["{0}: {1}".format(k, v) for k, v in n.items()]))

## g) prikazati nekretnine (Top30) koje imaju:
##    ▪ najveći broj soba unutar nekretnine,
##    ▪ najveću kvadraturu (samo za stanove),
##    ▪ najveću površinu zemljišta (samo za kuće)

In [ ]:
for n in db.top_broj_soba():
    print(" | ".join(["{0}: {1}".format(k, v) for k, v in n.items()]))

### KOMENTAR: Prva mesta u izlistavanju nize zauzimaju objekti sa greskom u podacima - najpre jer su kvadrature pogresno unete ili jer su kuce pogresno kategorisane kao stanovi.

In [ ]:
for n in db.top_kvadratura_stanovi():
    print(" | ".join(["{0}: {1}".format(k, v) for k, v in n.items()]))

### KOMENTAR: Neka od prvih mesta u izlistavanju nize potencijalno zauzimaju objekti sa netacno unetim podatkom o povrsini zemljista.

In [ ]:
for n in db.top_povrsina_zemljista():
    print(" | ".join(["{0}: {1}".format(k, v) for k, v in n.items()]))

In [ ]:
db.close()

# Task 3: data visualisation.

In [ ]:
import matplotlib.pyplot as plt
from db.mysql import MysqlDAO
db = MysqlDAO('nekretnine')

## a) 10 najzastupljenijih delova Beograda koji imaju najveći broj nekretnina u ponudi (i u sekciji za prodaju, i u sekciji za iznajmljivanje, zbirno)

In [ ]:
fig, ax = plt.subplots()
delovi_grada, brojevi_nekretnina = zip(*[(elem['deo_grada'], elem['broj_nekretnina']) for elem in db.top_delovi_beograda()])
ukupan_broj_nekretnina_u_beogradu = db.broj_nekretnina_u_beogradu()
ax.bar(delovi_grada, brojevi_nekretnina)
ax.set_title("Top 10 delova Beograda sa najvecim brojem nekretnina u ponudi")
ax.set_xlabel("deo grada")
ax.set_ylabel("broj nekretnina")

fig.set_size_inches(18, 3)
for deo, broj in zip(delovi_grada, brojevi_nekretnina):
    ax.annotate("{} ({:.1%})".format(broj, broj / ukupan_broj_nekretnina_u_beogradu), xy=(deo, broj - 100))

## b) Broj stanova za prodaju prema kvadraturi, u celoj Srbiji (do 35 kvadrata, 36-50, 51-65, 66-80, 81-95, 96-110, 111 kvadrata i više)

In [ ]:
fig, ax = plt.subplots()
kvadrature, brojevi_nekretnina = zip(*db.kategorije_kvadrature().items())
ukupan_broj_stanova_za_prodaju = db.broj_stanova_za_prodaju()
ax.bar(kvadrature, brojevi_nekretnina)
ax.set_title("Broj stanova za prodaju prema kvadraturi u Srbiji")
ax.set_xlabel("kvadratura")
ax.set_ylabel("broj nekretnina")

fig.set_size_inches(18, 3)
for kvadratura, broj in zip(kvadrature, brojevi_nekretnina):
    ax.annotate("{} ({:.1%})".format(broj, broj / ukupan_broj_stanova_za_prodaju), xy=(kvadratura, broj - 500))

## c) Broj izgrađenih nekretnina po dekadama (1951-1960, 1961-1970, 1971-1980, 1981-1990, 1991-2000, 2001-2010, 2011-2020), a obuhvatiti i sekcije za prodaju i za iznajmljivanje.

In [ ]:
fig, ax = plt.subplots()
dekade, brojevi_nekretnina = zip(*db.nekretnine_po_dekadama().items())
ax.bar(dekade, brojevi_nekretnina)
ax.set_title("Broj nekretnina u ponudi po dekadama u kojima su izgradjene")
ax.set_xlabel("dekada")
ax.set_ylabel("broj nekretnina")

fig.set_size_inches(18, 3)
for dekada, broj in zip(dekade, brojevi_nekretnina):
    ax.annotate(str(broj), xy=(dekada, broj - 200))

In [ ]:
db.close()

# Task 4: regression.

# Task 5: classification.

# Finalize.

In [ ]:
db.close()